In [156]:
import numpy as np
from scipy.optimize import root_scalar

# ==========================
# Constants from the paper
# ==========================
Tc = 647.096  # [K] Critical temperature of water
pc = 22.064e6  # [Pa] Critical pressure of water

# Molar masses [kg/mol]
M_LiBr = 0.08685
M_H2O = 0.018015268
# Coefficients for pressure equation (Eq. 1)
a = [-2.41303e2,1.91750e7,-1.75521e8,3.25430e7,3.92571e2,-2.12626e3,1.85127e8,1.91216e3]
m = [3, 4, 4, 8, 1, 1, 4, 6]
n = [0, 5, 6, 3, 0, 2, 6, 0]
t = [0, 0, 0, 0, 1, 1, 1, 1]
# Coefficients for water vapor pressure (Eq. 28)
a_ps = [-7.85951783, 1.84408259, -11.7866497,
        22.6807411, -15.9618719, 1.80122502] 
b_ps = [1.0, 1.5, 3.0, 3.5, 4.0, 7.5]
# ===========================================
# Saturation pressure of pure water, p_s(T)
# ===========================================
def ps_water(T):
    T=float(T)
    theta = 1 - T / Tc
    ln_ps = (Tc / T) * sum(a * theta**b for a, b in zip(a_ps, b_ps))
    return pc * np.exp(ln_ps)

# ===========================================
# Pressure equation of LiBr-H2O: p(T, x)
# ===========================================
def p_libr(T, x):
    q = T - sum(
        a[i] * x**m[i] * (0.4 - x)**n[i] * (T / Tc)**t[i]
        for i in range(8)
    )
    return ps_water(q)

# ===========================================
# Solve for x at given T and p
# ===========================================
def find_x_given_T_p(T, p_target):
    T = float(T)
    p_target = float(p_target)
    def objective(x):
        if x <= 0 or x >= 0.74:
            return 1e6
        return p_libr(T, x) - p_target

    a, b = 0.001, 0.73
    fa = objective(a)
    fb = objective(b)

    # Check if a solution is physically possible
    if np.sign(fa) == np.sign(fb):
        return None  # no sign change = no root

    result = root_scalar(objective, bracket=[a, b], method='bisect')

    # ❗ Additional guard to avoid edges
    if result.converged and 0.01 < result.root < 0.72:
        return x_to_w(result.root)
    else:
        return None  # discard if root is on edge



# ===========================================
# Convert molar fraction x → mass fraction w
# ===========================================
def x_to_w(x):
    m1 = x * M_LiBr
    m2 = (1 - x) * M_H2O
    return m1 / (m1 + m2)

from CoolProp.CoolProp import PropsSI

MLiBr = 0.08685      # kg/mol
MH2O = 0.018015      # kg/mol
T_c = 647.096        # K
T_0 = 221.0          # K
h_c = 37548.5        # J/mol

# Coefficients
a_h = [
    2.27431, -7.99511, 385.239, -16394, -422.562,
    0.113314, -8.33474, -17383.3, 6.49763, 3245.52,
    -13464.3, 39932.2, -258877, -0.00193046, 2.80616,
    -40.4479, 145.342, -2.74873, -449.743, -12.1794,
    -0.00583739, 0.233910, 0.341888, 8.85259, -17.8731,
    0.0735179, -0.000179430, 0.00184261, -0.00624282, 0.00684765
]
m_h = [1,1,2,3,6,1,3,5,4,5,5,6,6,1,2,2,2,5,6,7,1,1,2,2,2,3,1,1,1,1]
n_h = [0,1,6,6,2,0,0,4,0,4,5,5,6,0,3,5,7,0,3,1,0,4,2,6,7,0,0,1,2,3]
t_h = [0,0,0,0,0,1,1,1,2,2,2,2,2,3,3,3,3,3,3,3,4,4,4,4,4,4,5,5,5,5]

def mass_to_molar_fraction(w):
    return (w / MLiBr) / (w / MLiBr + (1 - w) / MH2O)

def h_LiBr(T, w):
    x = mass_to_molar_fraction(w)
    h_w_kg = PropsSI('H', 'T', T, 'Q', 0, 'Water')  # J/kg
    h_w_mol = h_w_kg * MH2O                        # J/mol
    second = 0.0
    for i in range(30):
        second += a_h[i] * x**m_h[i] * abs(0.4 - x)**n_h[i] * (T_c / (T - T_0))**t_h[i]
    # Total molar enthalpy [J/mol]
    h_molar = (1 - x) * h_w_mol + h_c * second
    M_mix = x * MLiBr + (1 - x) * MH2O
    h_mass = h_molar / M_mix  # J/kg
    return h_mass

s_c = 79.3933      # J/mol·K
s_ai = [
    1.53091, -4.52564, 698.302, -21666.4, -1475.33,
    0.0847012, -6.59523, -29533.1, 0.00956314, -0.188679,
    9.31752, 5.78104, 13893.1, -17176.2, 415.108,
    -55564.7, -0.00423409, 30.5242, -1.67620, 14.8283,
    0.00303055, -0.0401810, 0.149252, 2.59240, -0.177421,
    -0.0000699650, 0.000605007, -0.00165228, 0.00122966
]
s_mi = [1,1,2,3,6,1,3,5,1,2,2,4,5,5,6,6,1,3,5,7,1,1,1,2,3,1,1,1,1]
s_ni = [0,1,6,6,2,0,0,4,0,0,4,0,4,5,2,5,0,4,0,1,0,2,4,7,1,0,1,2,3]
s_ti = [0,0,0,0,0,1,1,1,2,2,2,2,2,2,2,2,3,3,3,3,4,4,4,4,4,5,5,5,5]

def mass_to_molar_fraction(w):
    return (w / MLiBr) / (w / MLiBr + (1 - w) / MH2O)

def s_LiBr(T, w):
    x = mass_to_molar_fraction(w)

    # Get entropy of pure water (J/kg·K) and convert to J/mol·K
    s_w_kgK = PropsSI('S', 'T', T, 'Q', 0, 'Water')  # J/kg·K
    s_w_molK = s_w_kgK * MH2O                      # J/mol·K
    second = sum(
        ai * x**mi * abs(0.4 - x)**ni * (T_c / (T - T_0))**ti
        for ai, mi, ni, ti in zip(s_ai, s_mi, s_ni, s_ti)
    )

    # Molar entropy of the solution [J/mol·K]
    s_molar = (1 - x) * s_w_molK + s_c * second
    M_mix = x * MLiBr + (1 - x) * MH2O
    s_mass = s_molar / M_mix  # J/kg·K
    return s_mass

from scipy.optimize import root_scalar
def find_T_given_h_x(h_target, x_mass, T_bounds=(273.15, 473.15)):
    """
    Invert the h_LiBr function to solve for T given enthalpy and mass fraction.
    h_target: enthalpy [J/kg]
    x_mass: LiBr mass fraction
    T_bounds: bounds for T in Kelvin
    """
    h_target=float(h_target)
    x_mass=float(x_mass)
    def objective(T):
        try:
            return h_LiBr(T, x_mass) - h_target
        except Exception:
            return 1e6  # fail safe if T is out of range

    result = root_scalar(objective, bracket=T_bounds, method='brentq')
    
    if result.converged:
        return result.root
    else:
        raise ValueError("Temperature solution did not converge.")



In [ ]:
from sympy import symbols, Eq, solve

# Define groups of symbols
def solve_system(Temperature_evaporator,P_gascooler,Temperature_after_gascooler,Water_line_temperature,Generator_temperature,Adsorber_temperature,Condensor_temperature,Efficiency_gas_cooler_1,Efficiency_heat_exchanger,Vars_evaporator_temperature):

    solutions=dict()
    misc_equations=T_e_tcrs,n_c,n_t,rp,W_turbine,W_compressor,Q_e_tcrs,COP,efficiency,P2s,s2s,h2s,P_e_tcrs,P5s,s5s,h5s,T5s,mw1,E_gc1,E_shx,T_generator,P_generator,Q_generator,T_adsorber,P_adsorber,Q_adsorber,T_e_vars,P_e_vars,Q_e_vars,T_condensor,P_condensor,Q_condensor,w_strong,w_weak,m_strong,m_weak,m_ra,W_input_tcrs,n_exergy_tcrs,n_exergy_vars,n_exergy_total,COP_tcrs,COP_comb=symbols('T_e_tcrs,n_c,n_t,rp,W_turbine,W_compressor,Q_e_tcrs,COP,efficiency,P2s,s2s,h2s,P_e_tcrs,P5s,s5s,h5s,T5s,' \
    'mw1,E_gc1,E_shx,T_generator,P_generator,Q_generator,T_adsorber,P_adsorber,Q_adsorber,T_e_vars,P_e_vars,Q_e_vars,T_condensor,P_condensor,Q_condensor,w_strong,w_weak,m_strong,m_weak,m_ra,W_input_tcrs,n_exergy_tcrs,n_exergy_vars,n_exergy_total,COP_tcrs,COP_comb')
    P=symbols('P:25')
    h=symbols('h:25')
    x=symbols('x:25')
    w=symbols('w:25')
    s=symbols('s:25')
    m=symbols('m:25')
    T=symbols('T:25')
    all_symbols = T + P +w+ x +s+m +h+(misc_equations)

# Define the equations
    #point 1
    equation = [
        Eq(T[1],Temperature_evaporator),
        # Eq(T_e_tcrs,Temperature_evaporator),
        Eq(x[1],1),
        Eq(m[1],1)
    ]
    solutions.update(solve(equation, all_symbols) )
    equation=[
        Eq(P[1],PropsSI('P','T',solutions[T[1]],'Q',solutions[x[1]],'CO2')),
        Eq(h[1],PropsSI('H','T',solutions[T[1]],'Q',solutions[x[1]],'CO2')),
        Eq(s[1],PropsSI('S','T',solutions[T[1]],'Q',solutions[x[1]],'CO2')),  
              ]
    solutions.update(solve(equation, all_symbols) )
    
    #point  2s
    equation=[
        Eq(P2s,P_gascooler),
        Eq(s2s, solutions[s[1]])
    ]
    solutions.update(solve(equation, all_symbols))

    equation=[
        Eq(h2s,PropsSI('H','P',solutions[P2s],'S',solutions[s2s],'CO2')),
        #evaporator
        Eq(P_e_tcrs,solutions[P[1]]),
        Eq(rp,solutions[P2s]/solutions[P[1]]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
         Eq(n_c,0.815+ 0.022*solutions[rp]-0.0041*(solutions[rp]**2)+0.0001*(solutions[rp]**3))
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 2
    equation=[
        Eq(P[2],P_gascooler),
        Eq(m[2],1),
        Eq(h[2],solutions[h[1]]+(solutions[h2s]-solutions[h[1]])/solutions[n_c])
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(T[2],PropsSI('T','P',solutions[P[2]],'H',solutions[h[2]],'CO2')),
        Eq(s[2],PropsSI('S','P',solutions[P[2]],'H',solutions[h[2]],'CO2'))
    ]
    solutions.update(solve(equation, all_symbols) )

      #point 16
    equation=[
        Eq(T[16],Water_line_temperature),
        Eq(x[16],0),
    ]
    solutions.update(solve(equation, all_symbols) )
    
    equation=[
        Eq(h[16],PropsSI('H','T',solutions[T[16]],'Q',solutions[x[16]],'Water')),
        Eq(s[16],PropsSI('S','T',solutions[T[16]],'Q',solutions[x[16]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 17
    equation=[
        Eq(T[17],Water_line_temperature),
        Eq(x[17],1),
    ]
    solutions.update(solve(equation, all_symbols) )
    
    equation=[
        Eq(h[17],PropsSI('H','T',solutions[T[17]],'Q',solutions[x[17]],'Water')),
        Eq(s[17],PropsSI('S','T',solutions[T[16]],'Q',solutions[x[17]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 3
    equation=[
        Eq(m[3],1),
        Eq(P[3],P_gascooler),
        Eq(T[3],solutions[T[2]]-Efficiency_gas_cooler_1*(solutions[T[2]]-solutions[T[16]])),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h[3],PropsSI('H','T',solutions[T[3]],'P',solutions[P[3]],'CO2')),
        Eq(s[3],PropsSI('S','T',solutions[T[3]],'P',solutions[P[3]],'CO2')),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(mw1,(solutions[h[2]]-solutions[h[3]])/(solutions[h[17]]-solutions[h[16]])),
        Eq(m[16],mw1),
        Eq(m[17],mw1),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 4
    equation=[
        Eq(m[4],1),
        Eq(P[4],P_gascooler),
        Eq(T[4],Temperature_after_gascooler),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(s[4],PropsSI('S','T',solutions[T[4]],'P',solutions[P[4]],'CO2')),
        Eq(h[4],PropsSI('H','T',solutions[T[4]],'P',solutions[P[4]],'CO2')),
    ]
    solutions.update(solve(equation, all_symbols) )

    # point 5s
    equation=[
        Eq(s5s,solutions[s[4]]),
        Eq(P5s,solutions[P_e_tcrs]),
        Eq(T5s,Temperature_evaporator),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h5s,PropsSI('H','S',solutions[s5s],'P',solutions[P5s],'CO2')),
        # turbine efficiency
        Eq(n_t,0.6)
    ]
    solutions.update(solve(equation, all_symbols) )

    # point 5
    equation=[
        Eq(m[5],1),
        Eq(P[5],solutions[P_e_tcrs]),
        Eq(h[5],solutions[h[4]]-solutions[n_t]*(solutions[h[4]]-solutions[h5s]))
    ]
    solutions.update(solve(equation, all_symbols) )
    
    equation=[
        Eq(T[5],PropsSI('T','P',solutions[P[5]],'H',solutions[h[5]],'CO2')),
        Eq(s[5],PropsSI('S','P',solutions[P[5]],'H',solutions[h[5]],'CO2')),
    ]
    solutions.update(solve(equation, all_symbols) )



    equation=[
        Eq(T_e_vars,Vars_evaporator_temperature)
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(P_e_vars,PropsSI('P','T',solutions[T_e_vars],'Q',0,'Water')),
        Eq(T_adsorber,Adsorber_temperature),
        Eq(T_generator,Generator_temperature),
        Eq(T_condensor,Condensor_temperature),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(P_adsorber,solutions[P_e_vars]),
        Eq(P_condensor,PropsSI('P','T',solutions[T_condensor],'Q',0,'Water')),
        Eq(P_generator,P_condensor),
    ]
    solutions.update(solve(equation, all_symbols) )

    # point 13
    equation=[
        Eq(T[13],solutions[T_generator]),
        Eq(P[13],solutions[P_generator]),
    ]
    solutions.update(solve(equation, all_symbols) )
    equation=[
        Eq(h[13],PropsSI('H','P',solutions[P[13]],'T',solutions[T[13]],'Water')),
        Eq(s[13],PropsSI('S','P',solutions[P[13]],'T',solutions[T[13]],'Water')),
    ]
    solutions.update(solve(equation, all_symbols) )
    
    #point 14
    equation=[
        Eq(T[14],solutions[T_condensor]),
        Eq(P[14],solutions[P_condensor]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h[14],PropsSI('H','T',solutions[T[14]],'Q',0,'Water')),
        Eq(s[14],PropsSI('S','T',solutions[T[14]],'Q',0,'Water')),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(T[15],solutions[T_e_vars]),
        Eq(P[15],solutions[P_e_vars]),
        Eq(h[15],solutions[h[14]]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(s[15],PropsSI('S','H',solutions[h[15]],'P',solutions[P[15]],'Water'))
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 6
    equation=[
        Eq(T[6],solutions[T_e_vars]),
        Eq(P[6],solutions[P_e_vars]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h[6],PropsSI('H','T',solutions[T[6]],'Q',1,'Water')),
        Eq(s[6],PropsSI('S','T',solutions[T[6]],'Q',1,'Water')),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 7
    equation=[
        Eq(T[7],solutions[T_adsorber]),
        Eq(P[7],solutions[P_adsorber]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(w[7],find_x_given_T_p(solutions[T[7]],solutions[P[7]])),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h[7],h_LiBr(solutions[T[7]],solutions[w[7]])),
        Eq(s[7],s_LiBr(solutions[T[7]],solutions[w[7]])),
        Eq(w_strong,solutions[w[7]]),
    ]
    solutions.update(solve(equation, all_symbols) )
    
    #point 8
    equation=[
        Eq(w[8],solutions[w[7]]),
        Eq(h[8],solutions[h[7]]),
        Eq(T[8],solutions[T[7]]),
        Eq(h[8],solutions[h[7]]),
        Eq(s[8],solutions[s[7]]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(w[9],solutions[w_strong]),
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 10
    equation=[
        Eq(T[10],solutions[T_generator]),
        Eq(P[10],solutions[P_generator]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(w[10],find_x_given_T_p(solutions[T[10]],solutions[P[10]])),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h[10],h_LiBr(solutions[T[10]],solutions[w[10]])),
        Eq(s[7],s_LiBr(solutions[T[10]],solutions[w[10]])),
        Eq(w_weak,solutions[w[10]]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(w[11],solutions[w_weak]),
        Eq(w[12],solutions[w_weak]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(Q_generator,solutions[mw1]*(solutions[h[17]]-solutions[h[16]]))
    ]
    solutions.update(solve(equation, all_symbols) )

    #point 11
    equation=[
        Eq(T[11],solutions[T[10]]-Efficiency_heat_exchanger*(solutions[T[10]]-solutions[T[8]])),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
        Eq(h[11],h_LiBr(solutions[T[11]],solutions[w[11]])),
        Eq(s[11],s_LiBr(solutions[T[11]],solutions[w[11]]))
    ]
    solutions.update(solve(equation, all_symbols) )
    
    equation=[
        Eq(h[12],solutions[h[11]]),
        Eq(s[12],solutions[s[11]]),
    ]
    solutions.update(solve(equation, all_symbols) )

    equation=[
      Eq(m_strong,m_ra+m_weak),
      Eq(solutions[w[7]]*m_strong,solutions[w[12]]*m_weak),
      Eq(m_strong*h[9]+solutions[Q_generator],m_weak*solutions[h[10]]+m_ra*solutions[h[13]]),
      Eq(m_weak*(solutions[h[10]]-solutions[h[11]]),m_strong*(h[9]-solutions[h[8]]))
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    equation=[
        Eq(m[13],solutions[m_ra]),
        Eq(m[14],solutions[m_ra]),
        Eq(m[15],solutions[m_ra]),
        Eq(m[6],solutions[m_ra]),
        Eq(m[7],solutions[m_strong]),
        Eq(m[8],solutions[m_strong]),
        Eq(m[9],solutions[m_strong]),
        Eq(m[10],solutions[m_weak]),
        Eq(m[11],solutions[m_weak]),
        Eq(m[12],solutions[m_weak]),
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    equation=[
        Eq(Q_condensor,solutions[m[13]]*solutions[h[13]]-solutions[m[14]]*solutions[h[14]]),
        Eq(Q_adsorber,solutions[m[6]]*solutions[h[6]]+solutions[m[12]]*solutions[h[12]]-solutions[m[7]]*solutions[h[7]]),
        Eq(Q_e_vars,solutions[m[6]]*solutions[h[6]]-solutions[m[15]]*solutions[h[15]])
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #point 9
    equation=[
        Eq(T[9],find_T_given_h_x(solutions[h[9]],solutions[w[9]]))
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )
    equation=[
        Eq(s[9],s_LiBr(solutions[T[9]],solutions[w[9]]))
    ]
    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    #values to find
    equation=[
        Eq(Q_e_tcrs,solutions[m[1]]*(solutions[h[1]]-solutions[h[5]])),
        Eq(W_compressor,solutions[m[1]]*(solutions[h[2]]-solutions[h[1]])),
        Eq(W_turbine,solutions[m[1]]*(solutions[h[4]]-solutions[h[5]])),
        Eq(W_input_tcrs,W_compressor-W_turbine),
        Eq(n_exergy_tcrs,Q_e_tcrs*(T[0]/(Temperature_evaporator+5)-1)/W_input_tcrs),
        Eq(T[0],298),
        Eq(COP_tcrs,Q_e_tcrs/W_input_tcrs),
        Eq(COP_comb,(solutions[Q_e_vars]+Q_e_tcrs)/W_input_tcrs),
        Eq(n_exergy_total,(Q_e_tcrs*(T[0]/(Temperature_evaporator+5)-1)+solutions[Q_e_vars]*(T[0]/(solutions[T_e_vars]+5)-1))/W_input_tcrs)
    ]

    solutions.update(solve(equation, all_symbols,dict=True)[0] )

    print(solutions)


In [161]:
solve_system(-25+273.15,9846000,40+273.15,100+273.15,90+273.15,35+273.15,273.15+35,0.8,0.7,7+273.15)

{T1: 248.150000000000, m1: 1.00000000000000, x1: 1.00000000000000, P1: 1682738.12720116, h1: 437055.401286796, s1: 1973.20884889544, P2s: 9846000.00000000, s2s: 1973.20884889544, P_e_tcrs: 1682738.12720116, h2s: 519542.909179935, rp: 5.85117781599001, n_c: 0.823389413708116, P2: 9846000.00000000, h2: 537235.834165643, m2: 1.00000000000000, T2: 395.647176808726, s2: 2018.67492360906, T16: 373.150000000000, x16: 0.0, h16: 419166.162892887, s16: 1307.21114214667, T17: 373.150000000000, x17: 1.00000000000000, h17: 2675569.88441946, s17: 7354.11914570156, P3: 9846000.00000000, T3: 377.649435361745, m3: 1.00000000000000, h3: 512065.857671119, s3: 1953.54395860725, m16: 0.0111549082526310, m17: 0.0111549082526310, mw1: 0.0111549082526310, P4: 9846000.00000000, T4: 313.150000000000, m4: 1.00000000000000, h4: 315535.449525928, s4: 1365.00989079318, P5s: 1682738.12720116, T5s: 248.150000000000, s5s: 1365.00989079318, h5s: 286130.829833738, n_t: 0.600000000000000, P5: 1682738.12720116, h5: 297892